In [ ]:

import torch
import os
import matplotlib.pyplot as plt
import numpy as np
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data import DataLoader
import random
import cv2
from PIL import Image

SEED = 123
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED) 

In [ ]:

BATCH_SIZE = 64
INPUT_SIZE = (256, 256)

def count_files_in_directory(directory):
    total_files = 0
    for root, dirs, files in os.walk(directory):
        total_files += len(files)
    return total_files

main_data_dir = r"C:\Users\Josh\Desktop\CUDA\skindiseases"
train_dir = os.path.join(main_data_dir, "train")
val_dir = os.path.join(main_data_dir, "val")
test_dir = os.path.join(main_data_dir, "test")

train_files = count_files_in_directory(train_dir)
val_files = count_files_in_directory(val_dir)
test_files = count_files_in_directory(test_dir)

print(f"Training Dataset: {train_files}")
print(f"Validation Dataset: {val_files}")
print(f"Test Dataset: {test_files}")

In [ ]:

# Load the training dataset to calculate mean and std, and get class labels
train_dataset = datasets.ImageFolder(root=train_dir)
class_n = list(train_dataset.class_to_idx.keys())  # Automatically retrieves class names from folders
print("Class to label mapping:", train_dataset.class_to_idx)

In [ ]:

class KMeansSegmentation:
    def __init__(self, n_clusters=8, overlay_alpha=0.5):
        self.n_clusters = n_clusters
        self.overlay_alpha = overlay_alpha

    def __call__(self, img):
        # Convert PIL image to NumPy array
        img_np = np.array(img)

        # Reshape the image to a 2D array of pixels
        pixel_values = img_np.reshape((-1, 3))
        pixel_values = np.float32(pixel_values)

        # Define criteria and apply K-means
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
        _, labels, centers = cv2.kmeans(pixel_values, self.n_clusters, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

        # Convert centers back to 8-bit values and reshape labels to the original image shape
        centers = np.uint8(centers)
        segmented_image = centers[labels.flatten()]
        segmented_image = segmented_image.reshape(img_np.shape)

        # Convert segmented image to grayscale to create a mask
        gray_segmented = cv2.cvtColor(segmented_image, cv2.COLOR_RGB2GRAY)
        _, mask = cv2.threshold(gray_segmented, 1, 255, cv2.THRESH_BINARY)

        # Blend the segmented regions back onto the original image
        img_segmented = cv2.addWeighted(img_np, 1 - self.overlay_alpha, segmented_image, self.overlay_alpha, 0)

        # Convert back to PIL Image
        return Image.fromarray(img_segmented)


In [ ]:

class CLAHETransform:
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clip_limit = clip_limit
        self.tile_grid_size = tile_grid_size

    def __call__(self, img):
        # Convert PIL Image to NumPy array
        img_np = np.array(img)
        
        # Apply CLAHE on each channel independently if it's a color image
        if len(img_np.shape) == 3:
            channels = cv2.split(img_np)
            clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
            channels = [clahe.apply(channel) for channel in channels]
            img_np = cv2.merge(channels)
        else:
            # Apply CLAHE on grayscale images
            clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
            img_np = clahe.apply(img_np)

        # Convert back to PIL Image
        img_clahe = Image.fromarray(img_np)
        return img_clahe


In [ ]:
import numpy as np
from torchvision import transforms

MEAN = (0.6181, 0.4643, 0.4194)
STD = (0.1927, 0.1677, 0.1617)


# Define transformations
transform_train = transforms.Compose([
    transforms.Resize(INPUT_SIZE),
    CLAHETransform(clip_limit=2.0, tile_grid_size=(8, 8)),
    transforms.RandomResizedCrop(227, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5), 
    transforms.RandomApply([KMeansSegmentation(n_clusters=3, overlay_alpha=0.5)], p=0.3),
    transforms.RandomApply([transforms.RandomRotation(degrees=(-20, 20))], p=0.5),
    transforms.RandomChoice([
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        transforms.RandomGrayscale(p=0.2)
    ]),
    transforms.RandomApply([transforms.RandomAffine(degrees=5, translate=(0.02, 0.02), shear=2)], p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

transform_val_test = transforms.Compose([
    transforms.Resize(INPUT_SIZE),
    CLAHETransform(clip_limit=2.0, tile_grid_size=(8, 8)),
    transforms.CenterCrop((227,227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])
# Load the datasets with the new transforms
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform_train)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform_val_test)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform_val_test)

# Create DataLoaders for each dataset
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Get a batch of images and labels from the DataLoader
images, labels = next(iter(train_loader))
print(images.shape)  

In [ ]:

# Function to unnormalize the image for visualization
def unnormalize(image, mean, std):
    # Convert the tensor to a NumPy array and transpose dimensions to (H, W, C)
    image = image.numpy().transpose((1, 2, 0))  
    
    # Unnormalize by reversing the mean and std normalization
    image = (image * std) + mean  
    
    # Clip values to be between 0 and 1 for valid image display
    image = np.clip(image, 0, 1)  
    return image


# Visualize a batch of images from the train_loader
def visualize_loader(loader, mean, std, class_names, num_images=6):
    # Get a batch of images
    data_iter = iter(loader)
    images, labels = next(data_iter)  

    # Plot the images
    plt.figure(figsize=(12, 8))
    for i in range(num_images):
        plt.subplot(2, 3, i+1)
        image = unnormalize(images[i], mean, std)  
        plt.imshow(image)
        plt.title(f"Class: {class_names[labels[i]]}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Use this function to visualize a batch of images
visualize_loader(train_loader, mean=MEAN, std=STD, class_names=class_n)

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

model = models.alexnet(weights=None)  


model.classifier = nn.Sequential(
    *list(model.classifier.children())[:3]  
)


weight_path = r""  
state_dict = torch.load(weight_path, weights_only=True)


new_state_dict = {}
for k, v in state_dict.items():
    new_key = k.replace("model.", "") if k.startswith("model.") else k
    if new_key in model.state_dict().keys():
        new_state_dict[new_key] = v


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


model.load_state_dict(new_state_dict, strict=False)  


for param in model.parameters():
    param.requires_grad = False


model.eval()
print(model)

# from torchsummary import summary
# input_size = (3, 227, 227)
# summary(model, input_size=input_size, device=str(device))

In [ ]:
import torch
import numpy as np

def extract_features(data_loader, model, device):
    features = []
    labels = []
    
    model.to(device)  # Move the model to the GPU
    model.eval()  # Set the model to evaluation mode
    
    with torch.no_grad():  # Disable gradient computation
        for images, target_labels in data_loader:
            images = images.to(device) 
            target_labels = target_labels.to(device)  

            # Extract features using the model
            output_features = model(images)
            
            # Move features back to the CPU and convert to NumPy arrays
            features.append(output_features.cpu().numpy()) 
            labels.append(target_labels.cpu().numpy())
    
    # Concatenate features and labels from all batches
    features = np.vstack(features)
    labels = np.concatenate(labels)
    return features, labels

train_features, y_train = extract_features(train_loader, model, device)
val_features, y_val = extract_features(val_loader, model, device)
test_features, y_test = extract_features(test_loader, model, device)


# Make sure the extracted features are in a format compatible with XGBoost
print("Train features shape:", train_features.shape)
print("Validation features shape:", val_features.shape)
print("Test features shape:", test_features.shape)

In [ ]:
import optuna
import plotly
from xgboost import XGBClassifier
import json
import tqdm as notebook_tqdm

def objective(trial):

    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.06, log=True), 
        'max_depth': trial.suggest_int('max_depth', 20, 30), 
        'n_estimators': trial.suggest_int('n_estimators', 300, 500),  
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 3),  
        'subsample': trial.suggest_float('subsample', 0.5, 0.7),  
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.7), 
    }


    xgb_model = XGBClassifier(
        objective='multi:softmax',
        num_class=len(class_n),
        random_state=SEED,
        eval_metric='merror',
        tree_method='hist',
        device='cuda',  
        early_stopping_rounds=20,
        **params,
    )

    xgb_model.fit(
        train_features, y_train,
        eval_set=[(val_features, y_val)],
        verbose=False,
    )

    accuracy = xgb_model.score(val_features, y_val)
    return accuracy


sampler = optuna.samplers.TPESampler(seed=SEED)


study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=300)

best_params = study.best_params
print("\nBest hyperparameters found:", best_params)

best_value = study.best_value
print("\nBest value found:", best_value)


optimized_xgb_model = XGBClassifier(
    objective='multi:softmax',
    num_class=len(class_n),
    random_state=SEED,
    eval_metric='merror',
    tree_method='hist',
    device='cuda',  
    early_stopping_rounds=20,  
    **best_params,
)

optimized_xgb_model.fit(
    train_features, y_train,
    eval_set=[(val_features, y_val)],
    verbose=True,
)

# Save model and parameters
optimized_xgb_model.save_model('final_xgboost_model.json')
with open('best_hyperparameters.json', 'w') as f:
    json.dump(best_params, f)
print("Model and parameters saved successfully.")

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
from sklearn.metrics import accuracy_score

train_predictions = optimized_xgb_model.predict(train_features)
train_accuracy = accuracy_score(y_train, train_predictions)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

val_predictions = optimized_xgb_model.predict(val_features)
val_accuracy = accuracy_score(y_val, val_predictions)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

test_predictions = optimized_xgb_model.predict(test_features)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')